## Use Inbuilt tools_condition for Conditional Routing

In [2]:
#Important Libraries
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode, tools_condition


In [3]:
#Initialize the LLM
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

#Define a multiplication tool
def multiply(a: int, b: int) -> int:
    """ Multiplies two numbers."""
    
    return a * b


#Bind the LLM with tool
llm_with_tools = llm.bind_tools([multiply])

#Node that call the LLM with tools bound
def tool_calling_llm(state: MessagesState):
    """Node that calls the LLM with tools bound."""

    return{"messages": [llm_with_tools.invoke(state["messages"])]}


#Build the workflow graph
workflow = StateGraph(MessagesState)
workflow.add_node("tool_calling_llm", tool_calling_llm)
#Tool node for handling tool invokations
workflow.add_node("tools", ToolNode([multiply]))

#Define edges with conditional routing
workflow.add_edge(START, "tool_calling_llm")
#Add the conditional edge based on tool usage
workflow.add_conditional_edges("tool_calling_llm", tools_condition)
workflow.add_edge("tools", END)

#Compile the graph
graph = workflow.compile()


In [4]:
#Simulate invoking the graph
def simulate():

    user_input = {"messages": [("human", "Can you multiply 3 by 5?")]}
    result = graph.invoke(user_input)
    return result["messages"][-1].pretty_print()

print(simulate())

================================= Tool Message =================================
Name: multiply

15
None


## Custom conditional routing function example: Weather and Calculator nodes